In [1]:
import numpy as np
import pandas as pd

def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        # Считываем X-вектор (первая строка)
        x_vector = list(map(int, file.readline().strip().split()))
        
        # Считываем Y-вектор (вторая строка)
        y_vector = list(map(int, file.readline().strip().split()))
        
        # Считываем количество узлов интерполирования (третья строка)
        n = int(file.readline().strip())
    
    return x_vector, y_vector, n

# Пример использования
file_path = 'untitled.txt'  # Замените на ваш путь к файлу
X, Y, n = read_data_from_file(file_path)

print("X-вектор:", X)
print("Y-вектор:", Y)
print("Количество узлов интерполирования:", n)

X-вектор: [0, 2, 3, 5, 6, -1]
Y-вектор: [-1, 113, 381, 1754, 3029, -16]
Количество узлов интерполирования: 3


In [2]:
ier = -1

# Проверка на неубывание вектора X
if not np.all(np.diff(X) <= 0):
    ier = 3

# Проверка на совпадение размеров векторов X и Y
if len(X) != len(Y):
    ier = 2

# Если ier равен 3, сортируем X и переставляем элементы в Y
#if ier == 3:
    #sorted_indices = np.argsort(X)
   # X = X[sorted_indices]
   # Y = Y[sorted_indices]

print("X:", X)
print("Y:", Y)
print("ier:", ier)

X: [0, 2, 3, 5, 6, -1]
Y: [-1, 113, 381, 1754, 3029, -16]
ier: 3


In [3]:

def divided_differences(X, Y):
    n = len(X)
    # Создаем таблицу для разностных сумм
    table = np.zeros((n, n))
    table[:, 0] = Y  # Заполняем первый столбец значениями Y

    # Вычисляем разностные суммы
    for j in range(1, n):
        for i in range(n - j):
            table[i, j] = (table[i + 1, j - 1] - table[i, j - 1]) / (X[i + j] - X[i])

    return table

# Вычисляем разностные суммы
table = divided_differences(X, Y)
# Создаем DataFrame для удобного отображения
def show_divided_differences(axis, table):
    df = pd.DataFrame(table, columns=[f'Δ^{i}' for i in range(len(axis))],index = [f'X{i+1}' for i in range(len(axis))])
    df.insert(0, 'x', axis)
    print(df)
    return df

show_divided_differences(X, table)



    x     Δ^0     Δ^1         Δ^2        Δ^3       Δ^4       Δ^5
X1  0    -1.0    57.0   70.333333  13.833333  0.055556  0.013889
X2  2   113.0   268.0  139.500000  14.166667  0.041667  0.000000
X3  3   381.0   686.5  196.166667  14.041667  0.000000  0.000000
X4  5  1754.0  1275.0  140.000000   0.000000  0.000000  0.000000
X5  6  3029.0   435.0    0.000000   0.000000  0.000000  0.000000
X6 -1   -16.0     0.0    0.000000   0.000000  0.000000  0.000000


,x,Δ^0,Δ^1,Δ^2,Δ^3,Δ^4,Δ^5
X1,0,-1.0,57.0,70.333333,13.833333,0.055556,0.013889
X2,2,113.0,268.0,139.500000,14.166667,0.041667,0.000000
X3,3,381.0,686.5,196.166667,14.041667,0.000000,0.000000
X4,5,1754.0,1275.0,140.000000,0.000000,0.000000,0.000000
X5,6,3029.0,435.0,0.000000,0.000000,0.000000,0.000000
X6,-1,-16.0,0.0,0.000000,0.000000,0.000000,0.000000


In [4]:
# Проверяем значение в последнем столбце первой строки
if table[0][-1] == 0:
    ier = 1  # Если значение равно 0
else:
    ier = 0  # Если значение не равно 0

print("ier =", ier)
# Индекс элемента, который будем исключать
index_to_exclude = 0
#функция нахождения индекс строки с ошибкой, если таблица верна, то возвращает -1 
# def error_in_table_is_found(axis, table):
#     index = 0

#функция нахождения коэффициентов многочлена третьей степени на основе таблицы разделенных разностей
def find_coefficients(axis, table):
    coefficients = [0,0,0,0]
    # f0 - значение функции в точке X0, f0=f(X0)
    f0 = table[0][0]
    # f01 - значение разделенной разности 1-го порядка в точках X0,X1, f01=f(X0,X1)
    f01 = table[0][1]
    # f012 - значение разделенной разности 2-го порядка в точках X0,X1,X2 f012=f(X0,X1,X2)
    f012 = table[0][2]
    # f0123 - значение разделенной разности 3-го порядка в точках X0,X1,X2,X3 f0123=f(X0,X1,X2,X3)
    f0123 = table[0][3]
    coefficients[0] = f0 - f01 * axis[0] + f012 * axis[0] * axis[1] - f0123 * axis[0] * axis[1] * axis[2]
    coefficients[1] = f01 - f012 * (axis[0]+axis[1]) + f0123 * (axis[0]*axis[1]+axis[1]*axis[2]+axis[0]*axis[2])
    coefficients[2] = f012 - f0123 * (axis[0]+axis[1]+axis[2])
    coefficients[3] = f0123
    return coefficients 
#значение функции в точке с данными коэффициентами
def foo(x, coefficients):
    return coefficients[0]+coefficients[1]*x+coefficients[2]*x*x+coefficients[3]*x*x*x

# Цикл проверки
while ier == 0 and index_to_exclude < len(X):
    print(f' x = {X[index_to_exclude]} excluded')
    # Создаем новые векторы без элемента с индексом index_to_exclude
    new_X = np.delete(X, index_to_exclude)
    new_Y = np.delete(Y, index_to_exclude)
    # Строим таблицу разностных сумм
    table = divided_differences(new_X, new_Y)
    show_divided_differences(new_X, table)
    # Проверяем первую строку последнего столбца
    if table[0][-1] == 0:
        print(f"Номер строки с ошибкой: {index_to_exclude+1}")
        ier = 1  # Завершаем цикл, если условие выполнено
        print(find_coefficients(new_X, table))
        print(foo(3,find_coefficients(new_X, table)))
    # Увеличиваем индекс для следующей итерации
    index_to_exclude += 1

ier = 0
 x = 0 excluded
    x     Δ^0     Δ^1         Δ^2        Δ^3       Δ^4
X1  2   113.0   268.0  139.500000  14.166667  0.041667
X2  3   381.0   686.5  196.166667  14.041667  0.000000
X3  5  1754.0  1275.0  140.000000   0.000000  0.000000
X4  6  3029.0   435.0    0.000000   0.000000  0.000000
X5 -1   -16.0     0.0    0.000000   0.000000  0.000000
 x = 2 excluded
    x     Δ^0          Δ^1         Δ^2        Δ^3       Δ^4
X1  0    -1.0   127.333333  111.833333  14.055556  0.013889
X2  3   381.0   686.500000  196.166667  14.041667  0.000000
X3  5  1754.0  1275.000000  140.000000   0.000000  0.000000
X4  6  3029.0   435.000000    0.000000   0.000000  0.000000
X5 -1   -16.0     0.000000    0.000000   0.000000  0.000000
 x = 3 excluded
    x     Δ^0     Δ^1    Δ^2   Δ^3  Δ^4
X1  0    -1.0    57.0   98.0  14.0 -0.0
X2  2   113.0   547.0  182.0  14.0  0.0
X3  5  1754.0  1275.0  140.0   0.0  0.0
X4  6  3029.0   435.0    0.0   0.0  0.0
X5 -1   -16.0     0.0    0.0   0.0  0.0
Номер строки с